In [ ]:
from helper import (ParquetUnifiedStorage, dividend_yield_screen)
from fetchers import (fetch_bars, fetch_dv_ttm, fetch_fina_audit)
import tushare as ts
from moonshot import Moonshot

: 

In [4]:
start = datetime.date(2018, 11, 30)
end = datetime.date(2023, 11, 30)

store_path = data_home / "rw/bars.parquet"
bars_store = ParquetUnifiedStorage(store_path=store_path, fetch_data_func=fetch_bars)


In [5]:

barss = bars_store.load_data(start, end)

2025-10-21 17:28:00.653 | INFO     | helper:load_data:283 - 获取开始日期前缺失的数据: 2018-11-30 到 2023-11-30
2025-10-21 17:28:00.695 | INFO     | tushare:_connect:29 - Connected to server tushare:5290


ImportError: pyarrow is required for converting a pandas dataframe to Polars, unless each of its columns is a simple numpy-backed one (e.g. 'int64', 'bool', 'float32' - not 'Int64')

In [ ]:
import polars as pl

df = pl.read_parquet("/Users/aaronyang/workspace/data/ro/bars_1d_2005_2023.parquet").to_pandas()

pl.from_pandas(df)

: 

In [ ]:
from helper import qfq_adjustment
barass_fq = qfq_adjustment(df = barss , adj_factor_col = 'adj_factor')

In [ ]:
# barass_fq.to_parquet('barass_fq.parquet')

In [ ]:
barass_fq

In [ ]:
ms = Moonshot(barass_fq)

# 筛选股票池条件,注意加因子的时候有的因子是nan但是不能直接去掉对应的股票

In [ ]:
# 连续2年分红
# 股息率排名前500
# 十年无审计保留
# 市值大于50亿元

In [ ]:
daily_basic_after = pd.read_parquet('daily_after.parquet')

In [ ]:
daily_basic_after

In [ ]:
ms.append_factor(daily_basic_after, "total_mv", resample_method="last")

In [ ]:
ms.data

In [ ]:
# 看一下空值占比多少
ms.data['total_mv'].isna().mean()

In [ ]:
ms.append_factor(daily_basic_after, "dv_ttm", resample_method="last")

In [ ]:
ms.data

In [ ]:
ms.data['dv_ttm'].isna().mean()

In [ ]:
# 对比发现原本.parquet这列空值就有35%左右

In [ ]:
ms.append_factor(daily_basic_after, "dividend_yield_2y_avg", resample_method="last")

In [ ]:
ms.data['dividend_yield_2y_avg'].isna().mean()

In [ ]:
ms.append_factor(daily_basic_after, "DPR", resample_method="last")

In [ ]:
ms.data['DPR'].isna().mean()

In [ ]:
# 之前没有计算换手率的波动
df = daily_basic_after.sort_values(['asset', 'date'])  # 按股票和时间排序

# 计算每个股票的换手率（turnover_rate_f）的30日滚动标准差
df['turnover_rate_f_std'] = (
    df.groupby('asset')['turnover_rate_f']
    .rolling(window=20, min_periods=1)  # 可调：min_periods=1 表示数据不足30天时也计算
    .std()
    .reset_index(level=0, drop=True)  # 去掉 groupby 的多重索引
)

In [ ]:
ms.append_factor(df, "turnover_rate_f_std", resample_method="last")

In [ ]:
ms.data['turnover_rate_f_std'].isna().mean()

In [ ]:
ms.append_factor(daily_basic_after, "inv_pe_ttm_zscore_5y", resample_method="last")

In [ ]:
ms.data['inv_pe_ttm_zscore_5y'].isna().mean()

In [ ]:
# 之前没有算div相比上个月的增加还是减少了
# 按股票和日期排序（先确保顺序正确）
df = daily_basic_after.sort_values(['asset', 'date'])

# 获取20天前的 dividend_ttm 值
df['dividend_ttm_20d_ago'] = df.groupby('asset')['dividend_ttm'].shift(20)

# 判断是否增加
df['dividend_ttm_increase_1M'] = (
    df['dividend_ttm'] > df['dividend_ttm_20d_ago']
)

In [ ]:
ms.append_factor(df, "dividend_ttm_increase_1M", resample_method="last")

In [ ]:
ms.data['dividend_ttm_increase_1M'].isna().mean()

In [ ]:
holder = pd.read_parquet('holdernum.parquet')

In [ ]:
holder.rename(columns={"ts_code": "asset","z_score": "holder_zscore"}, inplace=True)

In [ ]:
final_audit_df = pd.read_parquet('audit_reserve_flag.parquet')

In [ ]:
final_audit_df

In [ ]:
final_audit_df.rename(columns={"ts_code": "asset"}, inplace=True)

In [ ]:
ms.append_factor(final_audit_df, "has_audit_reserve", resample_method="last")

In [ ]:
ms.append_factor(holder, "holder_zscore", resample_method="last")

In [ ]:
ms.data['has_audit_reserve'].isna().mean()

In [ ]:
ms.data['holder_zscore'].isna().mean()

In [ ]:
# holder_zscore空值也很多需要检查一下

In [ ]:
n_cashflow_act =pd.read_parquet('n_cashflow_act.parquet')

In [ ]:
n_cashflow_act.rename(columns={"ts_code": "asset"}, inplace=True)

In [ ]:
income_8q_zscore = pd.read_parquet('income_8q_zscore.parquet')

In [ ]:
income_8q_zscore.rename(columns={"ts_code": "asset"}, inplace=True)

In [ ]:
balancesheet_asset_profit = pd.read_parquet('balancesheet_asset_profit.parquet')

In [ ]:
balancesheet_asset_profit.rename(columns={"ts_code": "asset"}, inplace=True)

In [ ]:
ms.append_factor(n_cashflow_act, "n_cashflow_act", resample_method="last")

In [ ]:
ms.append_factor(income_8q_zscore, "profit_z_score", resample_method="last") 

In [ ]:
ms.append_factor(balancesheet_asset_profit, "undistr_porfit", resample_method="last") 

In [ ]:
ms.append_factor(balancesheet_asset_profit, "total_assets", resample_method="last") 

In [ ]:
ms.data['n_cashflow_act'].isna().mean()

In [ ]:
ms.data['profit_z_score'].isna().mean()

In [ ]:
ms.data['undistr_porfit'].isna().mean()

In [ ]:
ms.data['total_assets'].isna().mean()

In [ ]:
ms.data

In [ ]:
# 分2步screen
# 先筛选股票池，是4个条件

In [ ]:
ms.data.columns

In [ ]:
def stock_pool_filter(data: pd.DataFrame) -> pd.Series:
    """
    股票池初筛：筛选高质量分红股，满足以下4个条件：
    1. 每月股息率前 500 名
    2. 连续两年有分红（两年平均股息率 > 0）
    3. 无审计保留意见（has_audit_reserve 为 False）
    4. 总市值 >= 50 亿

    返回：
        pd.Series：筛选标记，值为 0 或 1，index 与 data 对齐
    """

    df = data.copy()

    # 1. 股息率前 top_n 名（每月）
    dv_rank = df.groupby(level="month")['dv_ttm'].transform(lambda x: x.rank(method='first', ascending=False))
    cond1 = dv_rank <= 500
    # 但是股息率本身就带很多空值

    # # 2. 连续两年有分红
    cond2 = df['dividend_yield_2y_avg'] > 0

    # # 3. 无审计保留意见
    cond3 = df['has_audit_reserve'] == False
    # 可以看一下是否有必要放宽这个条件

    # 4. 总市值 > 50亿
    # 总市值 （万元）
    cond4 = df['total_mv'] >= 500000
    #  
    # 逻辑与汇总
    flag =  cond1 & cond2 & cond3 & cond4
    
    return flag.astype(int)

In [ ]:
# 先用这4个测试一下，看一下会不会过多空值被过滤了
# 单独用股息率会剩 0.11左右
# 市值会去掉一半左右
# 审计剩0.83
# 连续两年分红剩0.496
# 对比之前的连续2年分红算法,之前剩余0.06305968114016934，但是时间周期不完全一样，看一下回测是否相似
# 现在全部筛完剩0.066

In [ ]:
ms.screen(stock_pool_filter, data=ms.data) 

In [ ]:
(ms.data['flag'] == 1).sum() / len(ms.data)

In [ ]:
# ms.data['flag'] = 1 # 重置一下

In [ ]:
ms.screen(stock_pool_filter, data=ms.data).calculate_returns().report()

In [ ]:
def stock_pool_filter(data: pd.DataFrame) -> pd.Series:
    """
    股票池初筛：筛选高质量分红股，满足以下4个条件：
    1. 每月股息率前 500 名
    2. 连续两年有分红（两年平均股息率 > 0）
    3. 无审计保留意见（has_audit_reserve 为 False）
    4. 总市值 >= 50 亿

    返回：
        pd.Series：筛选标记，值为 0 或 1，index 与 data 对齐
    """

    df = data.copy()

    # 1. 股息率前 top_n 名（每月）
    dv_rank = df.groupby(level="month")['dv_ttm'].transform(lambda x: x.rank(method='first', ascending=False))
    cond1 = dv_rank <= 500
    # 但是股息率本身就带很多空值

    # # 2. 连续两年有分红
    cond2 = df['dividend_yield_2y_avg'] > 0

    # # 3. 无审计保留意见
    cond3 = df['has_audit_reserve'] == False
    # 可以看一下是否有必要放宽这个条件

    # 4. 总市值 > 50亿
    # 总市值 （万元）
    cond4 = df['total_mv'] >= 500000
    #  
    # 逻辑与汇总
    flag =  cond1 & cond2 & cond3 & cond4
    
    return flag.astype(int)

In [ ]:
def basic_factor_screen(data: pd.DataFrame, top_n: int = 30) -> pd.Series:
    df = data.copy()

    # 只对上一层筛选通过的股票打分
    if 'flag' in df.columns:
        df = df[df['flag'] == 1].copy()

    factor_rank_info = {
        'dividend_yield_2y_avg': True,
        'profit_z_score': True,
        'holder_zscore': False,          # 反转因子，越小越好
        'turnover_rate_f_std': False,    # 反转因子，越小越好
    }
    # 注意： 正向因子越大越好，反向因子越小越好，总分越高越优
    
    def rank_func(s, is_positive):
        if not is_positive:
            s = -s
        r = s.rank(method='average', ascending=True) # 注意这里是越大越好
        r = r.fillna(0)  # 缺失值最差（得分最低）
        return r


    def zscore_func(s, is_positive):
        """
        截面标准分（Z-score）
        正向因子：值越大得分越高；
        反向因子：值越小得分越高（取负后再标准化）。
        """
        s = s.copy()
        if not is_positive:
            s = -s
    
        mean = s.mean(skipna=True)
        std = s.std(skipna=True)
        
        if pd.isna(std) or std == 0:
            # 若该月该因子无波动（std=0或全NaN），则返回0分
            return pd.Series(0, index=s.index)
    
        z = (s - mean) / std
        z = z.fillna(0)  # 缺失值给0分（中性）
        return z
    

    def quantile_signal(s, upper=0.2, lower=0.2):
        s = s.copy()
        if s.notna().sum() == 0:
            # 全部缺失，返回全 0
            return pd.Series(0, index=s.index)
        high_q = s.quantile(1 - upper)
        low_q = s.quantile(lower)
        res = s.apply(lambda x: 1 if x >= high_q else (-1 if x <= low_q else 0))
        res = res.fillna(0)
        return res

    def dividend_signal(s, upper=0.2, lower=0.2):
        s = s.copy()
        if s.notna().sum() == 0:
            return pd.Series(0, index=s.index, dtype=int)
    
        high_q = s.quantile(1 - upper)
        low_q = s.quantile(lower)
    
        res = s.apply(lambda x: -1 if x >= high_q else (-1 if x <= low_q else 0))
        return res.fillna(0).astype(int)
    


    
    df['undistr_ratio'] = np.where(df['total_assets'] == 0, np.nan,
                                   df['undistr_porfit'] / df['total_assets'])
    df['cf_ratio'] = np.where(df['total_assets'] == 0, np.nan,
                              df['n_cashflow_act'] / df['total_assets'])

    
    df['score_dv_ttm'] = df.groupby('month')['dv_ttm'].transform(dividend_signal)
    df['score_ep'] = df.groupby('month')['inv_pe_ttm_zscore_5y'].transform(quantile_signal)
    
    df['score_cf'] = df.groupby('month')['cf_ratio'].transform(
        lambda s: (s <= s.quantile(0.2)).astype(int) * -1
    )
    df['score_undistr'] = df.groupby('month')['undistr_ratio'].transform(
        lambda s: (s <= s.quantile(0.2)).astype(int) * -1
    )
    
    df['score_dividend_increase'] = df['dividend_ttm_increase_1M'].fillna(False).astype(int)

    for fac, is_positive in factor_rank_info.items():
        df[f"{fac}_score"] = (
            df.groupby('month')[fac]
              .transform(lambda s: zscore_func(s, is_positive))
        )
        
    score_cols = [f"{fac}_score" for fac in factor_rank_info] + [
        'score_dv_ttm', 'score_cf', 'score_undistr',
        'score_ep', 'score_dividend_increase'
    ]

    # score_cols = ['score_dv_ttm', 'score_cf', 'score_undistr','score_ep', 'score_dividend_increase']

    # 总分：越高越好
    df['total_score'] = df[score_cols].sum(axis=1)
    # 只在已通过筛选的股票中选 top_n，注意top-n这里rank重新变成了真正的排名，rank越小代表排名越高
    flag = (
        df.groupby('month')['total_score']
          .transform(lambda s: (s.rank(method='first', ascending=False) <= top_n).astype(int))
    )
    # 对齐回原索引（未筛选股票置 0）
    flag = flag.reindex(data.index).fillna(0).astype(int)

    return flag

In [ ]:
ms.screen(stock_pool_filter, data=ms.data)
ms.screen(basic_factor_screen, data=ms.data, top_n=30)
(ms.data['flag'] == 1).sum() / len(ms.data)
ms.calculate_returns()
ms.report()